In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

import matplotlib.pyplot as plt
%matplotlib inline 

# Generate dummy data
import numpy as np

# Tensorboard
tbCallBack = keras.callbacks.TensorBoard(log_dir='/Graph', histogram_freq=0,  
          write_graph=True, write_images=True)

# Generate dummy data
x_train = np.random.random((1000, 20))
y_train = np.random.randint(2, size=(1000, 1))
x_test = np.random.random((100, 20))
y_test = np.random.randint(2, size=(100, 1))

model = Sequential()

tbCallBack.set_model(model)

# https://stackoverflow.com/questions/47877475/keras-tensorboard-plot-train-and-validation-scalars-in-a-same-figure/48393723

model.add(Dense(64, input_dim=20, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

weights, biases = model.layers[0].get_weights()
#tf.summary.histogram("weights", W1)

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
          epochs=20,
          batch_size=128)

#plt.plot(history)

score = model.evaluate(x_test, y_test, batch_size=128)

Epoch 1/20
1000/1000 [==============================] - 0s 471us/step - loss: 0.7116 - acc: 0.5160
Epoch 2/20
1000/1000 [==============================] - 0s 44us/step - loss: 0.7110 - acc: 0.4640
Epoch 3/20
1000/1000 [==============================] - 0s 42us/step - loss: 0.7054 - acc: 0.5050
Epoch 4/20
1000/1000 [==============================] - 0s 35us/step - loss: 0.7012 - acc: 0.4940
Epoch 5/20
1000/1000 [==============================] - 0s 39us/step - loss: 0.6940 - acc: 0.5200
Epoch 6/20
1000/1000 [==============================] - 0s 45us/step - loss: 0.6982 - acc: 0.5200
Epoch 7/20
1000/1000 [==============================] - 0s 38us/step - loss: 0.6997 - acc: 0.4960
Epoch 8/20
1000/1000 [==============================] - 0s 41us/step - loss: 0.6974 - acc: 0.5110
Epoch 9/20
1000/1000 [==============================] - 0s 41us/step - loss: 0.6931 - acc: 0.5420
Epoch 10/20
1000/1000 [==============================] - 0s 41us/step - loss: 0.6936 - acc: 0.5170
Epoch 11/20
1000/1

In [ ]:
# https://stackoverflow.com/questions/42112260/how-do-i-use-the-tensorboard-callback-of-keras
